In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import talib
import os

import warnings
warnings.filterwarnings('ignore')

def analyze_stock(filepath, ticker=None, output_dir="output/task2", show=True):
    """
    Load stock data from Excel, calculate TA indicators, and save plots.
    
    Parameters:
    - filepath: Path to the Excel file
    - ticker: Stock symbol (used in plot titles). If None, inferred from filename
    - output_dir: Where to save plots
    - show: Whether to show plots
    """
    os.makedirs(output_dir, exist_ok=True)

    # Load Excel file
    df = pd.read_excel(filepath)
    df.drop(columns=['Dividends', 'Stock Splits'], inplace=True, errors='ignore')
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    df.sort_index(inplace=True)

    # Set ticker name
    if not ticker:
        ticker = os.path.basename(filepath).split("_")[0]

    # Calculate Indicators
    df['SMA_20'] = talib.SMA(df['Close'], timeperiod=20)
    df['RSI_14'] = talib.RSI(df['Close'], timeperiod=14)
    df['MACD'], df['MACD_signal'], df['MACD_hist'] = talib.MACD(df['Close'])

    # Save processed data
    df.to_csv(f"{output_dir}/{ticker}_with_indicators.csv")

    # --- PLOTS ---
    # 1. Close Price + SMA
    plt.figure(figsize=(12, 6))
    plt.plot(df['Close'], label='Close Price', color='black')
    plt.plot(df['SMA_20'], label='SMA 20', color='blue')
    plt.title(f"{ticker} Close Price & 20-Day SMA")
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"{output_dir}/{ticker}_sma.png")
    if show: plt.show(); plt.close()

    # 2. RSI
    plt.figure(figsize=(12, 4))
    plt.plot(df['RSI_14'], color='purple')
    plt.axhline(70, color='red', linestyle='--')
    plt.axhline(30, color='green', linestyle='--')
    plt.title(f"{ticker} RSI (14)")
    plt.tight_layout()
    plt.savefig(f"{output_dir}/{ticker}_rsi.png")
    if show: plt.show(); plt.close()

    # 3. MACD
    plt.figure(figsize=(12, 4))
    plt.plot(df['MACD'], label='MACD', color='navy')
    plt.plot(df['MACD_signal'], label='Signal Line', color='orange')
    plt.title(f"{ticker} MACD")
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"{output_dir}/{ticker}_macd.png")
    if show: plt.show(); plt.close()

    print(f"✅ Processed and saved plots for {ticker}.")

# Example usage:
analyze_stock("../data/yfinance_data/AAPL_historical_data.xlsx")


ModuleNotFoundError: No module named 'talib'

In [4]:
!pip install TA_Lib‑0.4.0‑cp310‑cp310‑win_amd64.whl

Defaulting to user installation because normal site-packages is not writeable


ERROR: TA_Lib\u20110.4.0\u2011cp310\u2011cp310\u2011win_amd64.whl is not a valid wheel filename.
